In [1]:
import pandas as pd
import numpy as np
import tushare as ts
import os 
from tqdm import tqdm
import glob
ts.set_token('ce1d8f98f48f0c92fa70a66528612510f350b82ea565bc1abd45b5ac')
debug = False
from datetime import datetime 
pro = ts.pro_api()

## 股票基础信息

In [2]:
stocklist = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [4]:
stocklist.head()

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129
2,000004.SZ,000004,国华网安,深圳,软件服务,19910114
3,000006.SZ,000006,深振业A,深圳,区域地产,19920427
4,000007.SZ,000007,全新好,深圳,其他商业,19920413


## 交易日历

In [6]:
trade_cal = pro.trade_cal(exchange='', start_date='20190101', end_date='20250205')
trade_cal = trade_cal[trade_cal.is_open==1]
trade_cal.head()

,exchange,cal_date,is_open,pretrade_date
0,SSE,20250205,1,20250127
9,SSE,20250127,1,20250124
12,SSE,20250124,1,20250123
13,SSE,20250123,1,20250122
14,SSE,20250122,1,20250121


## 行情数据

In [5]:
price_PD = []
for trade_date in tqdm(trade_cal.cal_date):
    df = pro.daily(trade_date=trade_date)
    price_PD.append(df)
    
price_PD = pd.concat(price_PD).reset_index(drop=True)

price_PD = price_PD.drop_duplicates().sort_values(['ts_code','trade_date']).reset_index(drop=True)
price_PD.to_csv(f'data/price_PD.csv', index=False)   


100%|██████████████████████████████████████████████████████████████████████████████| 1247/1247 [13:26<00:00,  1.55it/s]


## 重要指标

In [6]:
price_PD = []
for trade_date in tqdm(trade_cal.cal_date):
    df = pro.daily_basic(ts_code='', trade_date=trade_date, \
                     fields='ts_code,trade_date,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,\
                     ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv')
    price_PD.append(df)
    
price_PD = pd.concat(price_PD).reset_index(drop=True)

price_PD = price_PD.drop_duplicates().sort_values(['ts_code','trade_date']).reset_index(drop=True)
price_PD.to_csv(f'data/daily_basic.csv', index=False)   


100%|██████████████████████████████████████████████████████████████████████████████| 1247/1247 [18:50<00:00,  1.10it/s]
